In [4]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
import random

def ascii_to_bits(key):
    # Convert ASCII characters to bits
    return [int(bit) for char in key for bit in bin(ord(char))[2:].zfill(8)]

def alice_prepare_qubits(key_bits):
    # Use the key bits directly for qubit preparation
    bits = key_bits
    return bits

def encode_qubits(bits):
    # Encode qubits using the BBM92 protocol
    encoded_qubits = []
    for bit in bits:
        qc = QuantumCircuit(1, 1)
        if bit == 1:
            qc.x(0)  # Apply X gate for |1⟩ basis
        qc.h(0)      # Apply Hadamard gate
        encoded_qubits.append(qc)
    return encoded_qubits

def bob_measure_qubits(encoded_qubits):
    # Bob measures qubits randomly in the |0⟩ or |1⟩ basis
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for qc in encoded_qubits:
        qc.measure(0, 0)
        t_qc = transpile(qc, backend)
        qobj = assemble(t_qc)
        result = backend.run(qobj).result()
        counts = result.get_counts()
        measured_bit = int(max(counts, key=counts.get))
        measurements.append(measured_bit)
    return measurements

def alice_select_bases(num_qubits):
    # Alice randomly selects measurement bases for each qubit
    bases = [random.choice([0, 1]) for _ in range(num_qubits)]
    return bases

def compare_bases(alice_bases, bob_bases):
    # Compare Alice's and Bob's selected bases
    min_len = min(len(alice_bases), len(bob_bases))
    matching_bases = [alice_bases[i] == bob_bases[i] for i in range(min_len)]
    return matching_bases

def key_from_bits(key_bits, matching_bases):
    # Extract a key from the matching bits
    key = [key_bits[i] for i in range(len(key_bits)) if matching_bases[i]]
    return key

def encrypt_numeric_input(numeric_input, key):
    # Convert numeric input to a list of bits
    bits = [int(bit) for digit in numeric_input for bit in bin(int(digit))[2:].zfill(4)]

    # Convert the key to bits
    key_bits = ascii_to_bits(key)

    # Alice prepares qubits using the key
    alice_bits = alice_prepare_qubits(key_bits)

    # Alice encodes qubits using BBM92
    encoded_qubits = encode_qubits(alice_bits)

    # Bob measures qubits
    bob_bits = bob_measure_qubits(encoded_qubits)

    # Alice selects measurement bases
    alice_bases = alice_select_bases(len(bits))

    # Compare Alice's and Bob's bases
    matching_bases = compare_bases(alice_bases, bob_bits)

    # Extract key from matching bits
    key = key_from_bits(key_bits, matching_bases)

    # XOR the key with the original bits to encrypt
    encrypted_bits = [bit ^ key_bit for bit, key_bit in zip(bits, key)]

    # Convert encrypted bits back to a string
    encrypted_numeric = ''.join([str(int(''.join(map(str, encrypted_bits[i:i+4])), 2)) for i in range(0, len(encrypted_bits), 4)])

    return encrypted_numeric

def save_to_binary_file(data, filename):
    # Save data to a binary file
    with open(filename, 'wb') as file:
        file.write(bytes(data, 'utf-8'))

In [7]:
def main():
    numeric_input = "03085333740033689998770335880001003443440042032406832930323566646103031333640032703499570324600820603175333308034339997100348701717103169666558030000093430324068337003411016958034408640230309800026803200928228031929998460342035022603121349627033699997790343666630403182683191033196080580315666660203118333425031100002830331233319803005016048034689999730316068316103204000254034376663290343037609003371666769031940002460306999997403000001675033063332970324568321003170016014031283497500306134952903243666880032320242380315989618803204608045032438000560303434963903031016249030000112420317034971703266984047031509999410332135013103356350176032363499130334233307503301208146030243335220342780005403075072054030280003080314566663403490999790034626664320320334992803264408051031413120160327001662703229683313030623335660345033341703221168076032100165440321668328803197666499033753332730330562402003218683238032756666500310083202403050968251032730562190314800034403143152059033174001740318566667203233864051034756400500316707215003000009688033757280680337609605003305000193034366836220318866690503254999854030193494270324784803103133016441031952240620307166664003146096003032103498260320516022603414999653033553501470314834986003152016405033358002130319559206403160683184034373681980330231208803186016549034093503310342596003703348016917030000014250330601667103415288248032583500650301101622303241224136033740721620324361603103322920183030386668050314534966003180872021033486665400330433325403202333336031148161850312401636603129904110031326666300342522403203261464202031418642440337922401503425017004034476721780303992803003142349686033646882090304066406303371350259030206829030300165605403302128140033626641420337444015603271683263031530881980327176813903344350226031526666300324966649803140024094033413201480313599994903366016823031413122300311734978603360999994032053498770300000941303243984062033676835760349101721503328016808031189999590313500010503254349946031316831520324361612903477666359034766663630306999999603490999575033225521440312634978303270016633032566802190306035224203096600052033615600040330733317803039000245030202400390345153612803224016561034500641030336174422403208333314031373498370346496802603473800023031300163760300100010203250424000031357920350347674404003368920236031876801130313601645903303666478030000160400323668331603222824229033271520310311187220303233016623031637602080325199975403419999778031710165530316486424003263333194033376401620311668309903078000394032493498990330912012303196880081030866640160321934983203366528149034208081430319399998403142666769033554880290337433332603079488045030785680240305354411103211683311034066665770330933313903459448169032520800310334135021703209349803030823334590326799980903089667049034453328880315601636103123349787030526828690343608006303359683486033093500510312634970603349666593031513496770333653601403416666685031153496760334835025303113682947033306835580306300008203318872003033339998220335068350103090666818032380165960303164801903000003760031935680590330400016403130349702031676666990332735001903183080231031260163320323966665703188333366034560171230341166650603107683007032009280370347472025303002666728032693500370308301631903036016278032626664720324159201003337683565030773335130324734993503426512034031919120270333801673103439017084031078242690033689200590323901600303270333230033776663610332301669703335016889031406830260314057602203204424212034579998780302263205403455032233030210162920343266658303229448256030423496160330033306603251016699033518080400347858403203236992208032743500810319233332503212683212030106721470334481607603188048222034400169210319798419703001016161033543840590311849621303255016600034312960730300000769503362333200030818800450344033300503314456014033576834330325244823503000015992030099361190336001679103017682824031039601820300055207303264305090032453332420309499992303028333321030000077510342435031303079304033030230000250316633619603256683222031743498990349666647903428168042033720480450306300022803229080094031020162800300001660303317952253033710169280302433355303244333122031899998460317000026703483000119030110160890330948806203364666400034638641550322533349203360456148033299120130315068304003491464168034959996520332468336903181666823031713331800326422425503181333354033797760590312733331603364683528034594482140347637614303329333125031413120220305501632403488704059032386832520322933358403114999923031786832050325428812103317016734033459997020321101660203091080110030798560680317168313803353464066034373332270324734986003171672065032230080760305152008803316016784034759997450310466664803237333304033410167270327000003903118128227034280170040330133303203052666880033686835130349901720803163000085"

    key = "{@nato},~4`(^_^)"  # Replace with your ASCII key

    encrypted_numeric = encrypt_numeric_input(numeric_input, key)

    # Save the encrypted numeric data to a binary file
    save_to_binary_file(encrypted_numeric, "encrypted__file.bin")

    print("Encrypted Numeric:", encrypted_numeric)
    print("Original:", numeric_input)
    print("Encrypted data saved to 'encrypted_file.bin'")

In [8]:
if __name__ == "__main__":
    main()

C:\Users\darkn\AppData\Local\Temp\ipykernel_6620\1022634609.py:32: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = backend.run(qobj).result()


Encrypted Numeric: 4211141541412651012411713
Original: 030853337400336899987703358800010034434400420324068329303235666461030313336400327034995703246008206031753333080343399971003487017171031696665580300000934303240683370034110169580344086402303098000268032009282280319299984603420350226031213496270336999977903436666304031826831910331960805803156666602031183334250311000028303312333198030050160480346899997303160683161032040002540343766632903430376090033716667690319400024603069999974030000016750330633329703245683210031700160140312834975003061349529032436668800323202423803159896188032046080450324380005603034349639030310162490300001124203170349717032669840470315099994103321350131033563501760323634991303342333075033012081460302433352203427800054030750720540302800030803145666634034909997900346266643203203349928032644080510314131201603270016627032296833130306233356603450333417032211680760321001654403216683288031976664990337533327303305624020032186832380327566665003100832024030509682510327305621